# Lifehacks:
    
ls -lt   - узнать какой лог последний

sq    - посмотреть статус по job'ам   

less   shift+G  - перейти в конец файла

## Предварительные работы

In [ ]:
cd /hse/amironov/Task8

wget ** все необходимые файлы

# Bash

genconf -f dppc.gro -o b_64.gro -nbox 4 4 4

editconf -f b_64.gro -o b_64.pdb
editconf -f dppc.gro -o dppc.pdb

правим файл в ноутбуке:

[ molecules ]
; name	number
DPPC    **64**

Сначала делаем отступ в ячейке от липидов, потом оптимизируем геометрию, потом добавляем воду.
Если бы мы сначала добавили воду, то мы с гораздо меньшей вероятностью пришли бы к оптимальной геометрии молекулы.

editconf -f b_64.gro -o b_ec -d 0.5

grompp -f em -c b_ec -p b -o b_em -maxwarn 2
mdrun -deffnm b_em -v

## PyMol Analysis of lipid before system optimization

dppc:

<img src="dppc.png">

b_64:

<img src="b_64.png">

## Оптимизация геометрии

In [ ]:
# Bash

grompp -f em -c b_ec -p b -o b_em -maxwarn 2

mdrun -deffnm b_em -v

Максимальная длина уменьшилась с 2.02*10^-2 nm до 1.9*10^-6 nm

Максимальная сила уменьшилась с 4.38*10^5 до 6.17*10^2

- формула силового поля
- либо параболические либо еще более сложные взаимодействия, но номер атома постоянно меняется 


## Добавляем воду

In [ ]:
# Bash

genbox -cp b_em -p b -cs spc216 -o b_s
grompp -f pr -c b_s -p b -o b_pr -maxwarn 1
mdrun -deffnm b_pr -v

Взрыва системы не было

In [ ]:
# Bash

grompp -f pr -c b_empr -p b -o b_pr -maxwarn 1
mdrun -deffnm b_pr -v

## Переводим в pdb

In [ ]:
# Bash

editconf -f b_pr.gro -o b_pr.pdb
editconf -f b_s.gro -o b_s.pdb

## PyMol Analysis of lipid after system optimization

b_pr:

<img src="b_pr.png">

b_s:

<img src="b_s.png">

b_s, убрали воду:

<img src="b_s_without_water.png">

**Если сравнить последнюю картинку с первой (b_64), то заметим, что на самом деле изменений в структуре липидного бислоя не произошло!**

## Копируем файлы

In [ ]:
# Bash

cd ..
scp -r ./Task8 lom:_scratch/chem/
    
пароль pracchem

## Тестовое моделирование на суперкомпьтере

In [ ]:
# Bash

ssh lom
cd _scratch/chem/Task8
cp /home/users/golovin/progs/share/gromacs/top/residuetypes.dat .
grompp -f md -c b_pr -p b -o b_md -maxwarn 1
sbatch -n 4 -e error.log -o output.log -t 5 -p test impi /opt/ccoe/gromacs-5.0.4/build/bin/gmx_mpi mdrun -testverlet -deffnm b_md -v

job: 1662130

Смотрим на файл output.log после того, как job пропал из списка.

файл пустой!

смотрим на error.log:

<img src="error log.png">

Также выше в этом файле было указано, что GPU не удалось инициализировать.


## Основное моделирование на суперкомпьтере

In [ ]:
# Bash

sbatch -N1 --ntasks-per-node=2 -e error-gpu.log -o output.log -t 350 -p gpu impi /opt/ccoe/gromacs-5.0.4/build/bin/gmx_mpi mdrun -testverlet -deffnm  b_md -v

Задача №: 1662201  